In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

#### Limpeza e tratamento do conjunto de dados meteorológicos

Fonte dos dados de velocidade do vento: Open-Meteo API
https://open-meteo.com/

Velocidade do vento tomada a uma altura de 100m acima do nível do solo.

Coordenadas utilizadas na chamada à API do Open-Meteo:
- Latitude: -10.640669 
- Longitude: -41.496846

Coordenadas disponíveis à API do Open-Meteo:
- Latitude: -10.650263
- Longitude: -41.46515

Fonte das coordenadas: ANEEL & Google Mapas

Notas:
- As torres de geração eólica do Conjunto Eólico Umburanas se distribuem em uma faixa de comprimento igual a aproximadamente 10 km. A coordenada utilizada na chamada à API foi escolhida arbitrariamente como um ponto próximo à metade deste comprimento. 
- Contudo, como os modelos meteorológicos utilizados pela plataforma Open-Meteo possuem uma resolução espacial de 9 km, as coordenadas para os quais os dados retornados pela plataforma não correspondem exatamente à coordenada utilizada na chamada à API.
- A distância entre os geradores eólicos e a coordenada efetivamente utilizada varia de 3.5 km e 7.5 km, aproximadamente. O ideal seria utilizar dados referentes a um ponto mais próximo aos geradores para diminuir o erro de estimação da velocidade do vento.
- Ainda, teria sido ideal selecionar uma coordenada para cada torre de geração e criar um modelo preditivo para cada torre de geração. A escolha da utilização de dados meteorológicos referentes a uma única coordenada para estimar a geração de todo o conjunto eólico pode contribuir para o aumento do erro de previsão do modelo. Contudo, com uma precisão espacial de 9 km, não seria possível garantir que os dados meteorológicos corresponderiam exatamente às coordenadas de cada torre de geração.


In [2]:
# Abrindo o arquivo descarregado da Open-Meteo API
df_tempo = pd.read_csv("..\Dados brutos\OPEN METEO\open-meteo-10.65S41.47W1000m.csv")

In [3]:
df_tempo.head()

,time,temperature_2m (°C),wind_speed_100m (km/h),wind_direction_100m (°),Unnamed: 4,Unnamed: 5,latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,2021-01-01T00:00,20.5,23.2,83,NaN,NaN,-10.650263,-41.46515,1000.0,-10800.0,America/Bahia,-3.0
1,2021-01-01T01:00,19.9,26.3,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01T02:00,18.9,25.6,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01T03:00,18.2,25.6,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01T04:00,17.9,24.5,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Criando uma função para reformatar a coluna que contém as datas e horários
from datetime import datetime
def formatDateTime(date_time):
    date_time = date_time.split("T")
    date, time = date_time
    date_time = date + "\t" + time
    return(datetime.strptime(date_time,"%Y-%m-%d\t%H:%M"))
print(formatDateTime("2021-01-01T01:00"))

2021-01-01 01:00:00


In [5]:
# Renomeando as colunas do DataFrame
df_tempo = df_tempo[["time","temperature_2m (°C)", "wind_speed_100m (km/h)", "wind_direction_100m (°)"]]
df_tempo.rename(columns={
    "time":"data_hora",
    "temperature_2m (°C)":"temperatura",
    "wind_speed_100m (km/h)":"velocidade_vento",
    "wind_direction_100m (°)":"direcao_vento"
}, inplace=True)
df_tempo.head()

,data_hora,temperatura,velocidade_vento,direcao_vento
0,2021-01-01T00:00,20.5,23.2,83
1,2021-01-01T01:00,19.9,26.3,100
2,2021-01-01T02:00,18.9,25.6,92
3,2021-01-01T03:00,18.2,25.6,92
4,2021-01-01T04:00,17.9,24.5,93


In [6]:
df_tempo.describe()

,temperatura,velocidade_vento,direcao_vento
count,24840.000000,24840.000000,24840.000000
mean,21.208023,22.393965,108.062480
std,3.922094,7.114559,42.860161
min,11.800000,0.000000,2.000000
25%,18.200000,18.300000,93.000000
50%,20.700000,23.400000,103.000000
75%,24.100000,27.300000,112.000000
max,34.000000,49.700000,360.000000


Amplitude da temperatura no local selecionado:
    de 11.3°C a 33.6°C
    
Amplitude da velocidade do vento no local selecionado (a uma altura de 100m acima do nível do solo):
    de 0 km/h a 49.7 km/h
    
Direção do vento: de 2° a 360°.

In [7]:
# Reformatando as datas e horários
df_tempo["data_hora"] = df_tempo["data_hora"].apply(formatDateTime)
df_tempo.head()
localidade = "umburanas-ba"

# Salvando os dados formatados
write_path = os.path.join(r"..\Dados tratados","tempo - " + localidade+ ".csv")
df_tempo.to_csv(write_path, encoding = "utf-8", index=False)